<a href="https://colab.research.google.com/github/namtoptall/AI-SUMMIT_test/blob/main/milestone_project_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huấn luyện mô hình : Model training
**Lưu ý ⚠ em huấn luyện mô hình trên google colab nên nếu muốn chạy lại toàn bộ thì cần set up những tài khoản sau :**
> tài khoản kaggle và kaggle API cho việc download bộ dữ liệu

> tài khoản google Drive để kết nối với kho dữ liệu cá nhân



### kết nối với google drive và kaggle

Với kaggle

In [1]:
from google.colab import files
files.upload()  # Thực hiện tải tệp kaggle.json lên từ máy tính (phải có tài khoản kaggle nha)

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"namdz242221","key":"de869a9575f6ac51974bca65557d88a7"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Mount google drive content vào trong notebook

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!kaggle datasets download -d namdz242221/dsp305x-01-a-vn-dataset

Dataset URL: https://www.kaggle.com/datasets/namdz242221/dsp305x-01-a-vn-dataset
License(s): MIT
... resuming from 491057122 bytes (64899025 bytes left) ...
100% 529M/530M [00:05<00:00, 21.2MB/s]
100% 530M/530M [00:05<00:00, 12.6MB/s]


unzip the dataset

In [5]:
!unzip dsp305x-01-a-vn-dataset.zip -d /content/dataset


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
file #22439:  bad zipfile offset (local header sig):  350382236
file #22440:  bad zipfile offset (local header sig):  350391820
file #22441:  bad zipfile offset (local header sig):  350402124
file #22442:  bad zipfile offset (local header sig):  350411221
file #22443:  bad zipfile offset (local header sig):  350420984
file #22444:  bad zipfile offset (local header sig):  350433326
file #22445:  bad zipfile offset (local header sig):  350442854
file #22446:  bad zipfile offset (local header sig):  350460322
file #22447:  bad zipfile offset (local header sig):  350470887
file #22448:  bad zipfile offset (local header sig):  350479348
file #22449:  bad zipfile offset (local header sig):  350489750
file #22450:  bad zipfile offset (local header sig):  350502338
file #22451:  bad zipfile offset (local header sig):  350512129
file #22452:  bad zipfile offset (local header sig):  350524022
file #22453:  bad zipfile offset (local header 

In [6]:
!ls /content/dataset

DSP305x_01-A_VN-dataset  test  train  val


check content trong tệp

In [7]:
!ls /content/dataset/train

 Bougainvillea	 Lotus	 'Peach blossom'  'Plum blossom'   Rose
 Chrysanthemum	 Orchid   Peony		   Plumeria	   Sunflower


In [8]:
import os
dataset_dir = "/content/dataset"
for root, dirs, files in os.walk(dataset_dir):
    print(root, "contains", len(files), "files.")


/content/dataset contains 0 files.
/content/dataset/DSP305x_01-A_VN-dataset contains 1 files.
/content/dataset/DSP305x_01-A_VN-dataset/train contains 0 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Sunflower contains 1 files.
/content/dataset/test contains 0 files.
/content/dataset/test/Plumeria contains 730 files.
/content/dataset/test/Bougainvillea contains 788 files.
/content/dataset/test/Orchid contains 742 files.
/content/dataset/test/Peony contains 738 files.
/content/dataset/test/Lotus contains 747 files.
/content/dataset/test/Peach blossom contains 726 files.
/content/dataset/test/Plum blossom contains 745 files.
/content/dataset/test/Rose contains 742 files.
/content/dataset/test/Sunflower contains 731 files.
/content/dataset/test/Chrysanthemum contains 735 files.
/content/dataset/val contains 0 files.
/content/dataset/val/Plumeria contains 735 files.
/content/dataset/val/Bougainvillea contains 799 files.
/content/dataset/val/Orchid contains 738 files.
/content/dataset

check shape của từng ảnh

In [9]:
from PIL import Image
import os
from collections import defaultdict

def check_unique_shapes(dataset_dir):
    # count the number of difference shapes
    shape_counts = defaultdict(int)

    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                try:
                    with Image.open(img_path) as img:
                        shape = img.size  # Lấy kích thước (chiều rộng, chiều cao) của hình ảnh
                        shape_counts[shape] += 1
                except Exception as e:
                    print(f"Error processing image {img_path}: {e}")

    return shape_counts

# Ví dụ sử dụng
dataset_dir = "/content/dataset"
unique_shapes = check_unique_shapes(dataset_dir)

# In ra các shape và số lần xuất hiện
for shape, count in unique_shapes.items():
    print(f"Shape {shape} appears {count} times.")


Shape (224, 224) appears 32335 times.


👉 shape của những ảnh trong dataset là 224,224

In [10]:
# xóa file metadata.json (chỉ có tác dụng khi upload data lên kaggle.)
!rm /content/dataset/dataset-metadata.json

rm: cannot remove '/content/dataset/dataset-metadata.json': No such file or directory


## một chút pre-check và labelling dataset

tải file .utils/ETL_functions.py từ thư mục đồ án

In [11]:
from google.colab import files
files.upload()  # Thực hiện tải ETL_functions.py lên từ máy tính

Saving ETL_functions.py to ETL_functions (1).py


{'ETL_functions (1).py': b'import os\r\nimport numpy as np\r\nimport pandas as pd\r\nimport tensorflow as tf\r\nfrom PIL import Image\r\nfrom sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix\r\nimport itertools\r\nimport matplotlib.pyplot as plt\r\nimport datetime\r\n# set the max image pixels to None\r\nImage.MAX_IMAGE_PIXELS = None\r\nimport os\r\nfrom PIL import Image, UnidentifiedImageError\r\n\r\n# Constants\r\nTARGET_IMG_SIZE = 256\r\n\r\nCLASS_LABELS = [\r\n    "Lotus", \r\n    "Chrysanthemum",\r\n    "Peony",\r\n    "Orchid",\r\n    "Sunflower",\r\n    "Bougainvillea",\r\n    "Rose",\r\n    "peach blossom",\r\n    "Plumeria",\r\n    "Plum blossom"\r\n]\r\n\r\ndef extract_image_info(image_path: str) -> dict:\r\n    """\r\n    Extract information from an image file.\r\n\r\n    :param image_path: Full path to the image file.\r\n    :return: A dictionary with extracted image information or None if the file is not a valid image.\r\n    """\r\n

In [12]:
import ETL_functions as etl
from ETL_functions import *

## Mở bộ dữ liệu hoa

In [13]:
train_path = "dataset/train"
test_path = "dataset/test"
val_path = "dataset/val"

In [14]:
train= etl.load_dataset(train_path)
test = etl.load_dataset(test_path)
val = etl.load_dataset(val_path)
train

,ImgPath,FileType,Width,Height,Ratio,Mode,Bands,Transparency,Animated,Class
0,/content/dataset/train/Plumeria/001051.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
1,/content/dataset/train/Plumeria/000209.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
2,/content/dataset/train/Plumeria/001867.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
3,/content/dataset/train/Plumeria/000990.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
4,/content/dataset/train/Plumeria/001429.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
...,...,...,...,...,...,...,...,...,...,...
17414,/content/dataset/train/Chrysanthemum/000676.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Chrysanthemum
17415,/content/dataset/train/Chrysanthemum/001947.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Chrysanthemum
17416,/content/dataset/train/Chrysanthemum/000480.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Chrysanthemum
17417,/content/dataset/train/Chrysanthemum/000224.jpg,jpg,224,224,1.0,RGB,R G B,False,False,Chrysanthemum


## Tạo model

#### thiết lập mixed_precision


In [17]:
# Enable mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")
mixed_precision.global_policy()

<DTypePolicy "mixed_float16">

In [16]:
train.shape, test.shape, val.shape

((17419, 10), (7424, 10), (7491, 10))

### Tạo tensorboard callbacks và train dữ liệu

In [18]:
import tensorflow as tf
import datetime
from tensorflow.keras.applications import EfficientNetB0, ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os

In [29]:
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
    """
    Creates a TensorBoard callback instance to store log files.

    Stores log files with the filepath:
      "dir_name/experiment_name/current_datetime/"

    Args:
      dir_name: target directory to store TensorBoard log files
      experiment_name: name of experiment directory (e.g. efficientnet_model_1)
    """
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir
    )
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

checkpoint_path = "model_checkpoints/.weights.h5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor="val_acc",
    save_best_only=True,
    save_weights_only=True,
    verbose=0
)


In [31]:
# Preprocessing steps can include resizing, normalization, and augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)


In [32]:
# Create iterators for train, validation, and test data
train_generator = datagen.flow_from_dataframe(train, x_col='ImgPath', y_col='Class', target_size=(224, 224))
val_generator = datagen.flow_from_dataframe(val, x_col='ImgPath', y_col='Class', target_size=(224, 224))
test_generator = datagen.flow_from_dataframe(test, x_col='ImgPath', y_col='Class', target_size=(224, 224))


Found 17419 validated image filenames belonging to 10 classes.
Found 7491 validated image filenames belonging to 10 classes.
Found 7424 validated image filenames belonging to 10 classes.


In [35]:
def build_model(base_model, num_classes):
    # Freeze the base model
    base_model.trainable = False

    # Define the input layer
    inputs = Input(shape=(224, 224, 3))

    # Pass the input through the base model
    x = base_model(inputs, training=False)

    # Global Average Pooling for downsampling the feature map
    x = GlobalAveragePooling2D()(x)

    # Flatten the output for the fully connected layers
    x = Flatten()(x)

    # Add a fully connected dense layer with ReLU activation
    x = Dense(128, activation="relu")(x)

    # Add a dropout layer for regularization
    x = Dropout(0.5)(x)

    # Final output layer with softmax activation for multi-class classification
    outputs = Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = Model(inputs, outputs)
    return model


In [36]:
# Number of classes (e.g., 10 classes for 10 flower types)
num_classes = 10

# Initialize the pre-trained models
efficientnet_model = build_model(EfficientNetB0(weights="imagenet", include_top=False), num_classes)
resnet_model = build_model(ResNet50(weights="imagenet", include_top=False), num_classes)
vgg16_model = build_model(VGG16(weights="imagenet", include_top=False), num_classes)

# Compile the models with optimizer, loss, and metrics
efficientnet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
resnet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
vgg16_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
# Train the models
print("Training EfficientNetB0 model...")

history_eff = efficientnet_model.fit(train_generator, epochs=10,
                                     validation_data=val_generator,
                                     callbacks=[model_checkpoint])

print("Training ResNet50 model...")

history_resnet = resnet_model.fit(train_generator,
                                  epochs=10,
                                  validation_data=val_generator,
                                  callbacks=[model_checkpoint])

print("Training VGG16 model...")
history_vgg = vgg16_model.fit(train_generator,
                              epochs=10,
                              validation_data=val_generator,
                              callbacks=[model_checkpoint])


Training EfficientNetB0 model...
Epoch 1/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 362s 658ms/step - accuracy: 0.1024 - loss: 2.3066 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


545/545 ━━━━━━━━━━━━━━━━━━━━ 289s 526ms/step - accuracy: 0.1075 - loss: 2.3024 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 3/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 323s 528ms/step - accuracy: 0.1059 - loss: 2.3025 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 4/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 285s 519ms/step - accuracy: 0.1064 - loss: 2.3025 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 5/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 344s 628ms/step - accuracy: 0.1091 - loss: 2.3022 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 6/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 290s 529ms/step - accuracy: 0.1077 - loss: 2.3023 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 7/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 322s 528ms/step - accuracy: 0.1047 - loss: 2.3026 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 8/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 292s 532ms/step - accuracy: 0.1052 - loss: 2.3026 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 9/10


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


545/545 ━━━━━━━━━━━━━━━━━━━━ 323s 534ms/step - accuracy: 0.1035 - loss: 2.3026 - val_accuracy: 0.1067 - val_loss: 2.3023
Epoch 10/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 346s 631ms/step - accuracy: 0.1046 - loss: 2.3025 - val_accuracy: 0.1067 - val_loss: 2.3023
Training ResNet50 model...
Epoch 1/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 327s 567ms/step - accuracy: 0.1147 - loss: 2.3406 - val_accuracy: 0.1315 - val_loss: 2.2589
Epoch 2/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 292s 528ms/step - accuracy: 0.1221 - loss: 2.2745 - val_accuracy: 0.1593 - val_loss: 2.2336
Epoch 3/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 318s 525ms/step - accuracy: 0.1361 - loss: 2.2620 - val_accuracy: 0.1412 - val_loss: 2.2455
Epoch 4/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 324s 528ms/step - accuracy: 0.1339 - loss: 2.2574 - val_accuracy: 0.1642 - val_loss: 2.2309
Epoch 5/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 345s 629ms/step - accuracy: 0.1371 - loss: 2.2544 - val_accuracy: 0.1617 - val_loss: 2.2207
Epoch 6/10
545/545 ━━━━━━━━━━━━━━━━━━━━ 293s 534ms/step - a

In [ ]:
def plot_history(histories, names):
    plt.figure(figsize=(14, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    for history in histories:
        plt.plot(history.history['accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(names, loc='upper left')

    # Plot loss
    plt.subplot(1, 2, 2)
    for history in histories:
        plt.plot(history.history['loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(names, loc='upper left')

    plt.show()

# Plotting the history for all models
plot_history([history_eff, history_resnet, history_vgg], ["EfficientNet", "ResNet50", "VGG16"])

In [ ]:
%tensorboard --logdir logs
